In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("lab07-JJAE") \
    .master("spark://1b1e9b41e03d:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [12]:
from JJAE.listener import TrafficListener

spark.streams.addListener(TrafficListener())

In [13]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import split, col

logs_schema = SparkUtils.generate_schema([("value", "string")])

logs_df = spark \
                .readStream \
                .schema(logs_schema) \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/strucutred_streaming_files/")


parsed_df = logs_df.select(
    split(col("value"), " \\| ").alias("parts")
).select(
    col("parts")[0].alias("date"),
    col("parts")[1].alias("type"),
    col("parts")[2].alias("description"),
    col("parts")[3].alias("server_node")
)

In [14]:
query = parsed_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime='2 seconds') \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination(10)

25/04/03 01:36:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dad58970-3f60-41a8-8aca-ef77e65f3df2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/03 01:36:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 04eb1d20-685a-416d-8da0-77c820caf4d6
-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----+----------------------------------------------+-------------+
|date               |type |description                                   |server_node  |
+-------------------+-----+----------------------------------------------+-------------+
|2025-04-02 18:45:10|ERROR|Authentication error                          |Server-node-2|
|2025-04-02 18:45:10|WARN |Configuration parameter missing, using default|Server-node-5|
|2025-04-02 18:45:10|INFO |Cache refreshed                               |Server-node-1|
|2025-04-02 18:45:10|WARN |Resource utilization above threshold          |Server-node-4|
|2025-04-02 18:45:10|WARN |High memory usage detected                    |Server-node-5|
|2025-04-02 18:45:10|INFO |Configuration loaded                          |Server-node-5|
|2025-04-02 18:45:10|ERROR|File not found         

False

In [ ]:
output = parsed_df.writeStream \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/data/strucutred_streaming_output/") \
    .option("checkpointLocation", "/home/jovyan/checkpoint") \
    .start()

output.awaitTermination(10)

25/04/03 01:37:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 2e1e0a6e-c338-4572-94fa-46c404379b32


25/04/03 01:37:07 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/batch_2_logs_20250402_193702.log to file:/home/jovyan/archive/home/jovyan/notebooks/data/strucutred_streaming_files/batch_2_logs_20250402_193702.log / skip moving file.


-------------------------------------------
Batch: 23
-------------------------------------------
+-------------------+-----+--------------------+-------------+
|               date| type|         description|  server_node|
+-------------------+-----+--------------------+-------------+
|2025-04-02 19:37:07| INFO|Data processing c...|Server-node-4|
|2025-04-02 19:37:07| INFO|Data processing c...|Server-node-3|
|2025-04-02 19:37:07| WARN|Timeout on extern...|Server-node-1|
|2025-04-02 19:37:07| INFO|Database connecti...|Server-node-1|
|2025-04-02 19:37:07| WARN|Connection attemp...|Server-node-2|
|2025-04-02 19:37:07|ERROR|  API request failed|Server-node-1|
|2025-04-02 19:37:07| WARN|Resource utilizat...|Server-node-2|
|2025-04-02 19:37:07|ERROR|Authentication error|Server-node-4|
+-------------------+-----+--------------------+-------------+

Query made progress: {
  "id" : "2e1e0a6e-c338-4572-94fa-46c404379b32",
  "runId" : "d1baf445-e903-4a05-9770-a5e4353682bb",
  "name" : null,
  "t

25/04/03 01:37:12 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/batch_3_logs_20250402_193707.log to file:/home/jovyan/archive/home/jovyan/notebooks/data/strucutred_streaming_files/batch_3_logs_20250402_193707.log / skip moving file.


...

-------------------------------------------
Batch: 32
-------------------------------------------
+-------------------+-----+--------------------+-------------+
|               date| type|         description|  server_node|
+-------------------+-----+--------------------+-------------+
|2025-04-02 19:37:52| INFO|Database connecti...|Server-node-4|
|2025-04-02 19:37:52| WARN|Connection attemp...|Server-node-4|
|2025-04-02 19:37:52| INFO|Application start...|Server-node-1|
|2025-04-02 19:37:52| INFO|Data processing c...|Server-node-2|
|2025-04-02 19:37:52|ERROR|Database connecti...|Server-node-5|
+-------------------+-----+--------------------+-------------+

-------------------------------------------
Batch: 22
-------------------------------------------
+-------------------+-----+--------------------+-------------+
|               date| type|         description|  server_node|
+-------------------+-----+--------------------+-------------+
|2025-04-02 19:37:52| INFO|Database connecti...

25/04/03 01:37:57 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/batch_12_logs_20250402_193752.log to file:/home/jovyan/archive/home/jovyan/notebooks/data/strucutred_streaming_files/batch_12_logs_20250402_193752.log / skip moving file.


-------------------------------------------
Batch: 33
-------------------------------------------
+-------------------+-----+--------------------+-------------+
|               date| type|         description|  server_node|
+-------------------+-----+--------------------+-------------+
|2025-04-02 19:37:57| WARN|Configuration par...|Server-node-5|
|2025-04-02 19:37:57|ERROR|Database connecti...|Server-node-5|
|2025-04-02 19:37:57| WARN|High memory usage...|Server-node-4|
|2025-04-02 19:37:57| INFO|Data processing c...|Server-node-5|
|2025-04-02 19:37:57| INFO|Scheduled task ex...|Server-node-5|
|2025-04-02 19:37:57|ERROR|Data corruption d...|Server-node-2|
|2025-04-02 19:37:57| WARN|Configuration par...|Server-node-3|
+-------------------+-----+--------------------+-------------+

-------------------------------------------
Batch: 23
-------------------------------------------
+-------------------+-----+--------------------+-------------+
|               date| type|         description

25/04/03 01:38:02 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/batch_13_logs_20250402_193757.log to file:/home/jovyan/archive/home/jovyan/notebooks/data/strucutred_streaming_files/batch_13_logs_20250402_193757.log / skip moving file.


-------------------------------------------
Batch: 24
-------------------------------------------
+-------------------+----+--------------------+-------------+
|               date|type|         description|  server_node|
+-------------------+----+--------------------+-------------+
|2025-04-02 19:38:02|WARN|High memory usage...|Server-node-4|
|2025-04-02 19:38:02|INFO|     Cache refreshed|Server-node-2|
|2025-04-02 19:38:02|WARN|Resource utilizat...|Server-node-5|
|2025-04-02 19:38:02|WARN|Timeout on extern...|Server-node-1|
|2025-04-02 19:38:02|INFO|Configuration loaded|Server-node-1|
+-------------------+----+--------------------+-------------+

-------------------------------------------
Batch: 34
-------------------------------------------
+-------------------+----+--------------------+-------------+
|               date|type|         description|  server_node|
+-------------------+----+--------------------+-------------+
|2025-04-02 19:38:02|WARN|High memory usage...|Server-node-